In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from pathlib import Path
import time
import random

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We need to install the transformers library first, if we use Google Colab.

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 3.3MB 23.0MB/s 
     |████████████████████████████████| 901kB 15.2MB/s 


From transformers, we need only the model itself, which is `DistilBertForSequenceClassification`; special optimizer that works with it, which is `AdamW`; and a pretrained tokenizer `DistilBertTokenizer` to feed our data correctly into the model. 

For the sake of saving space and performance, we use `DistilBert` in this notebook.

In [3]:
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
from transformers import DistilBertTokenizer
from transformers import get_linear_schedule_with_warmup


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


# Downloading and unpacking the data.

In [4]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn import preprocessing
 
df = pd.read_csv("drive/My Drive/NN/train/train.csv")
df.rename({'image':'image_name'},axis=1,inplace=True)
N = len(df['label_group'].unique())
le = preprocessing.LabelEncoder()
df['label_group'] = le.fit_transform(df['label_group'].values)
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))


df

,posting_id,image_name,image_phash,title,label_group,matches
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",7572,train_3386243561 train_3423213080
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,train_2288590299 train_3803689425
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,10509,train_2406599165 train_3342059966
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,train_3369186413 train_921438619
...,...,...,...,...,...,...
34245,train_4028265689,fff1c07ceefc2c970a7964cfb81981c5.jpg,e3cd72389f248f21,Masker Bahan Kain Spunbond Non Woven 75 gsm 3 ...,9735,train_2829161572 train_4028265689
34246,train_769054909,fff401691371bdcb382a0d9075dfea6a.jpg,be86851f72e2853c,MamyPoko Pants Royal Soft - S 70 - Popok Celana,7038,train_1463059254 train_769054909
34247,train_614977732,fff421b78fa7284284724baf249f522e.jpg,ad27f0d08c0fcbf0,KHANZAACC Robot RE101S 1.2mm Subwoofer Bass Me...,10537,train_4126022211 train_3926241003 train_232545...
34248,train_3630949769,fff51b87916dbfb6d0f8faa01bee67b8.jpg,e3b13bd1d896c05c,"Kaldu NON MSG HALAL Mama Kamu Ayam Kampung , S...",4242,train_3419392575 train_1431563868 train_363094...


In [7]:
df_train = pd.DataFrame()
t1 = []
t2 = []
same = []
c = 0
for i in range(df.shape[0]):
  mchs = df["matches"][i].split()
  ttl = df["title"][i]
  for j in range(min(len(mchs),5)):
    t1.append(ttl)
    t2.append(df.loc[df["posting_id"] == mchs[j],"title"].values[0])
    same.append(True)
  for j in range(10-min(len(mchs),5)):
    r = np.random.randint(0, df.shape[0])
    t1.append(ttl)
    t2.append(df.loc[r].title)
    same.append(df["label_group"][i]==df.loc[r].label_group)
  if i % 100 == 0:
    print(i)

df_train["t1"] = t1
df_train["t2"] = t2
df_train["same"] = same


34000
34100
34200


In [9]:
df_train["t1"] = t1
df_train["t2"] = t2
df_train["same"] = same

In [10]:
tr = df_train.loc[df_train["same"] == True]
tr
fl = df_train.loc[df_train["same"] == False].reset_index()
fl

,index,t1,t2,same
0,2,Paper Bag Victoria Secret,Audio Splitter U Shape Male to Dual Female Jac...,False
1,3,Paper Bag Victoria Secret,HC KACA MATA ANTI RADIASI HC 6789 / KACAMATA K...,False
2,4,Paper Bag Victoria Secret,SEPATU SLIP ON WANITA PRES,False
3,5,Paper Bag Victoria Secret,"Cartridge Filter Air / Sediment 10"" - Nano Fil...",False
4,6,Paper Bag Victoria Secret,iGlove Sarung Tangan Capacitive Smartphone & T...,False
...,...,...,...,...
227204,342495,FLEX TAPE PELAPIS BOCOR / ISOLASI AJAIB / ANTI...,RK GLOW / RK GLOW ORI/ RK GLOW PREMIUM / RK GL...,False
227205,342496,FLEX TAPE PELAPIS BOCOR / ISOLASI AJAIB / ANTI...,(BPOM)Parfum Garuda EAU TOILETTE LAVINA 5 Va(r...,False
227206,342497,FLEX TAPE PELAPIS BOCOR / ISOLASI AJAIB / ANTI...,Tradislim Pelangsing Tradisional Ramuan Putri ...,False
227207,342498,FLEX TAPE PELAPIS BOCOR / ISOLASI AJAIB / ANTI...,Makarizo Advisor Vitamax Hair Recovery 50ml,False


In [12]:
df_train[0:10]

,t1,t2,same
0,Paper Bag Victoria Secret,Paper Bag Victoria Secret,True
1,Paper Bag Victoria Secret,PAPER BAG VICTORIA SECRET,True
2,Paper Bag Victoria Secret,Audio Splitter U Shape Male to Dual Female Jac...,False
3,Paper Bag Victoria Secret,HC KACA MATA ANTI RADIASI HC 6789 / KACAMATA K...,False
4,Paper Bag Victoria Secret,SEPATU SLIP ON WANITA PRES,False
5,Paper Bag Victoria Secret,"Cartridge Filter Air / Sediment 10"" - Nano Fil...",False
6,Paper Bag Victoria Secret,iGlove Sarung Tangan Capacitive Smartphone & T...,False
7,Paper Bag Victoria Secret,Willy cardy/crop cardy/cardigan rajut/cardigan...,False
8,Paper Bag Victoria Secret,[READY!!!] LIP BALM KISS BEAUTY / TANAKO PEACH,False
9,Paper Bag Victoria Secret,Bestway 67002 Kasur Angin Double Biru [191cm x...,False


In [13]:
dataa = {}
data = [fl,tr]
labs = {}
for i in [0,1]:
  p = data[i]
  perc = np.random.rand(p.shape[0])
  perc = perc > 0.9
  p["test"] = perc
  dataa[i] = p
  labs[i] = i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Initializig the tokenizer from the pretrained `distilbert-base-uncased` model.

In [14]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True, padding_side="right")

In [15]:
print(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today."))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today.")))
tokenizer.encode_plus("This is the BERT tokenizer that we're going to use today.")

['this', 'is', 'the', 'bert', 'token', '##izer', 'that', 'we', "'", 're', 'going', 'to', 'use', 'today', '.']
[2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012]


{'input_ids': [101, 2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
PAD = '[PAD]'
PAD_ID = 0

batch_size = 16
validation_split = .3
shuffle_dataset = True
random_seed = 42

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

In [23]:
class IMDBDataSet(Dataset):
    def __init__(self, folders, pretrain_tokenizer, data, test=False):
        self.tokenizer = pretrain_tokenizer
        self.label_vocab = folders
        self.max_len = 128
        self.sign = False
        self.in_data= data

        if test:
            self.sign = True
        else:
            self.sign = False
            
        self.data = []
        
        self.load()
        
    def load(self):
        for label in self.label_vocab:
          
          p = self.in_data[self.label_vocab[label]].dropna()
          p = p.loc[p["test"]==self.sign]
          print(f'Reading {label} sentences...')
          for t1,t2 in zip(p["t1"],p["t2"]):
            text = self.tokenizer.encode(t1,t2, add_special_tokens = True, max_length=self.max_len, padding= "max_length", truncation = True, return_tensors = "pt")
            attention_mask = text > 0
            attention_mask = attention_mask.squeeze()
            torch_label = torch.tensor(self.label_vocab[label], dtype = torch.long)
              # append text, attention text and torch_label
            self.data.append((text.squeeze(), attention_mask, torch_label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]

In [24]:
train_data = IMDBDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa)

Reading 0 sentences...
Reading 1 sentences...


In [25]:
test_data = IMDBDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa, test=True)

Reading 0 sentences...
Reading 1 sentences...


In [26]:
train_data[0]

(tensor([  101,  3259,  4524,  3848,  3595,   102,  5746,  3975,  3334,  1057,
          4338,  3287,  2000,  7037,  2931,  2990,  1017,  1012,  1019,  7382,
         15488,  9307, 23025,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [27]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [28]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [29]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels= len(train_data.label_vocab), output_attentions = False, output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The DistilBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [30]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

epochs = 1
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [31]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
def multi_label_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    confusion_vector = rounded_preds / y
    
    true_positives = torch.sum(confusion_vector == 1,dtype=torch.float)
    false_positives = torch.sum(torch.isinf(confusion_vector),dtype=torch.float)
    false_negatives = torch.sum(confusion_vector == 0 ,dtype=torch.float)
    true_negatives = torch.sum(torch.isnan(confusion_vector),dtype=torch.float)

    accuracy = (true_positives+true_negatives) / (true_positives+true_negatives+false_negatives+false_positives)
    precision = (true_positives) / (true_positives+false_positives)
    recall = (true_positives) / (true_positives+false_negatives)
    f_score = 2*(precision*recall)/(precision+recall)
    return accuracy, precision, recall, f_score

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
# Taken from this tutorial: https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb
# The code was modified

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed_mins, elapsed_secs = epoch_time(t0, time.time())
            
            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_loader):>5,}.    Elapsed: {elapsed_mins:}m {elapsed_secs:}s.')

        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, attention_mask= b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.4f}")
    print("  Training epcoh took: {:}m {:}s".format(*epoch_time(t0, time.time())))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, attention_mask= b_input_mask)


        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        #print(label_ids)
        #print( np.argmax(logits, axis=1).flatten())
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.6f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}m {:}s".format(*epoch_time(t0, time.time())))

print("")
print("Training complete!")

  Batch 9,520  of  13,476.    Elapsed: 57m 51s.
  Batch 9,560  of  13,476.    Elapsed: 58m 5s.
  Batch 9,600  of  13,476.    Elapsed: 58m 20s.
  Batch 9,640  of  13,476.    Elapsed: 58m 35s.
  Batch 9,680  of  13,476.    Elapsed: 58m 49s.
  Batch 9,720  of  13,476.    Elapsed: 59m 4s.
  Batch 9,760  of  13,476.    Elapsed: 59m 18s.
  Batch 9,800  of  13,476.    Elapsed: 59m 33s.
  Batch 9,840  of  13,476.    Elapsed: 59m 48s.
  Batch 9,880  of  13,476.    Elapsed: 60m 2s.
  Batch 9,920  of  13,476.    Elapsed: 60m 17s.
  Batch 9,960  of  13,476.    Elapsed: 60m 31s.
  Batch 10,000  of  13,476.    Elapsed: 60m 46s.
  Batch 10,040  of  13,476.    Elapsed: 61m 1s.
  Batch 10,080  of  13,476.    Elapsed: 61m 15s.
  Batch 10,120  of  13,476.    Elapsed: 61m 30s.
  Batch 10,160  of  13,476.    Elapsed: 61m 44s.
  Batch 10,200  of  13,476.    Elapsed: 61m 59s.
  Batch 10,240  of  13,476.    Elapsed: 62m 14s.
  Batch 10,280  of  13,476.    Elapsed: 62m 28s.
  Batch 10,320  of  13,476.    Elaps

In [35]:
print("")
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0

# Evaluate data for one epoch
for batch in test_loader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, b_input_mask)


    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()
    print(label_ids)
    print( np.argmax(logits, axis=1).flatten())
    # Calculate the accuracy for this batch of test sentences.
    tmp_test_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    test_accuracy += tmp_test_accuracy

    # Track the number of batches
    nb_test_steps += 1

# Report the final accuracy for this test run.
print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
print("  Testing took: {:}m {:}s".format(*epoch_time(t0, time.time())))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0

In [37]:
def predict_same(model, s1, s2, voc):
    model.eval()
    text = tokenizer.encode(s1,s2, add_special_tokens = True, max_length=512, padding= "max_length", truncation = True, return_tensors = "pt")
    attention_mask = text > 0
    attention_mask = attention_mask.squeeze().to(device)
    text = text.to(device)
    bind = (text, attention_mask)
    #print(text, attention_mask)
    with torch.no_grad():
      outputs = model(bind[0], bind[1])[0]
    proba = outputs.detach().cpu().numpy()
    prediction, indices = torch.topk(torch.sigmoid(outputs), 1)
    indices = indices.detach().cpu().numpy()
    mojis = [voc[pred]for pred in indices[0]]
    return mojis, proba[0]


In [44]:
print(predict_same(model, "Sexy  underware","Unique pants", labs))

([1], array([-0.03071238,  0.09490123], dtype=float32))


In [ ]:
train = pd.read_csv("/content/drive/My Drive/NN/train.csv")
test = pd.read_csv("/content/drive/My Drive/NN/test.csv")

In [ ]:
post_ids = []
matches = []
buf = []
for i in range(test.shape[0]):
    post_ids.append(test.loc[i]['posting_id'])
    buf = str(test.loc[i]['posting_id'])
    for j in range(train.shape[0]):
        res = predict_same(model, test.loc[i]['title'],train.loc[j]['title'], labs)[0][0]
        if j % 10000 == 0:
            print(j)
        if res == 1:
            buf +=  " " + str(train.loc[j]['posting_id'])
        
    matches.append(buf)

In [ ]:
sub = pd.DataFrame()
sub["posting_id"] = post_ids
sub["matches"] = matches



In [36]:

import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/My Drive/NN/fine_tuned_model/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/drive/My Drive/NN/fine_tuned_model/


('/content/drive/My Drive/NN/fine_tuned_model/tokenizer_config.json',
 '/content/drive/My Drive/NN/fine_tuned_model/special_tokens_map.json',
 '/content/drive/My Drive/NN/fine_tuned_model/vocab.txt',
 '/content/drive/My Drive/NN/fine_tuned_model/added_tokens.json')

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = '/content/drive/My Drive/NN/model/'
model = DistilBertForSequenceClassification.from_pretrained(output_dir)
tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       